# Evaluating the Models

## This Juypter Notebook was created to easily load the models and set/change the directories to test the models, all in one place.

In [ ]:
# Allows evaluate.ipynb to access modules in "sibling" directories
import sys
sys.path.append("./") # Run this from parent directory

In [ ]:
import os
from database import Database
from models.altered_xception import AlteredXception
from models.faster_r_cnn import FasterRCNN
from models.rmac_model import RMACModel
from utils import grab_images_and_paths, grab_all_image_paths, resize_images
from models.query import query_image

## Loading The Models and Data
> Note: if you get any error surrounding a missing layer, a simple restart of the kernel should fix it.

In [ ]:
# NOTE: if the RMAC model needs to get descriptions of many different square regions, it will take a long time to load/initialize (roughly a minute).
ax_model = AlteredXception()
rmac_model = RMACModel(ax_model.model.get_layer("conv2d_3").output_shape[1:], 3)
rcnn_model = FasterRCNN()

In [ ]:
# Choose what models to evaluate, by including or discluding them from this array
models = [ax_model, rmac_model]

In [ ]:
# test_dir is the directory where our "database" of test images/frames and query images should exist. 
test_dir = "/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games"
frame_paths, frames = grab_images_and_paths(os.path.join(test_dir, "frames"))
query_image_paths = grab_all_image_paths(os.path.join(test_dir, "query_images"))
query_image_paths

In [ ]:
# Get the YouTube video "ground truth" from the IDs we picked, stored in a .txt file
ground_truth_ids = []
ground_truth_file_path = os.path.join(test_dir, "ground_truth.txt")
with open(ground_truth_file_path, "r") as ground_truth:
    for line in ground_truth.readlines():
        youtube_ground_truth_id = line.split("/")[-1].split(".")[0]
        ground_truth_ids.append(youtube_ground_truth_id)
ground_truth_ids

In [ ]:
# Initialize models to query from the evaluation database
database = Database(os.path.join(test_dir, "predictions"))
for model in models:
    model.database = database

In [ ]:
# Gathering predictions for each of the frames in the dataset, for each model
for model in models:
    print("Current model: ", model)
    if (isinstance(model, AlteredXception)) and len(database.prediction_image_paths) < len(frame_paths):
        model_predictions = model.predict_images(frames)
        database.store_predictions(model_predictions, frame_paths)
    if (isinstance(model, RMACModel)) and len(database.rmac_prediction_image_paths) < len(frame_paths):
        model_predictions = model.predict_images(frames)
        database.store_rmac_predictions(model_predictions, frame_paths)
    if (isinstance(model, FasterRCNN)) and len(database.object_predictions) < len(resized_frame_paths):
        model_predictions = model.predict_image_paths(resized_frame_paths)
        database.store_object_data(model_predictions, frame_paths)

In [ ]:
def post_process_predictions(predictions):
    model_preds = {}
    for model in predictions:
        video_id_preds = []
        for video_link in predictions[model].keys():
            video_id = video_link.split("/")[-1] # We only want the last part of the path
            video_id_preds.append(video_id)
        model_preds[model] = video_id_preds
    return model_preds

In [ ]:
results = {}
for query_image_path in query_image_paths:
    results[query_image_path] = {}
    results[query_image_path] = post_process_predictions(query_image(query_image_path, models, len(ground_truth_ids)))
results

In [ ]:
matches = {}
for query_image_path in query_image_paths:
    curr_results = results[query_image_path]
    matches[query_image_path] = {}
    for model in results[query_image_path]:
        num_matches = 0
        for id in ground_truth_ids:
            if id in results[query_image_path][model]:
                num_matches += 1
        matches[query_image_path][model] = num_matches
matches